In [1]:
import torch


torch.cuda.is_available()

True

In [2]:
import pathlib

model_file = "../data/ELYZA-japanese-Llama-2-7b-fast-instruct-q8_0.gguf"
# model_file = "../data/ELYZA-japanese-Llama-2-13b-fast-instruct-q8_0.gguf"
# model_file = "./ELYZA-japanese-Llama-2-7b-fast-instruct-q8_0.gguf"    # self-making
# model_file = "./ELYZA-japanese-CodeLlama-7b-instruct-q8_0.gguf"       # self-making
pathlib.Path(model_file).exists()

True

In [3]:
from llama_cpp import Llama

n_gqa = 8 if "70b" in model_file else 1
llm = Llama(model_path=model_file, n_gqa=n_gqa, n_gpu_layers=16)
llm

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 2 CUDA devices:
  Device 0: NVIDIA GeForce GTX 1080 Ti, compute capability 6.1, VMM: yes
  Device 1: NVIDIA GeForce GTX 1080 Ti, compute capability 6.1, VMM: yes
llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from ../data/ELYZA-japanese-Llama-2-7b-fast-instruct-q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = ELYZA-japanese-Llama-2-7b-fast-instruct
llama_model_loader: - kv   2:      general.source.huggingface.repository str              = elyza/ELYZA-japanese-Llama-2-7b-fast-...
llama_model_loader: - kv   3:                   llama.tensor_data_layout str              = Meta 

In [4]:
prompt = """富士山の高さは？正確に"""
# prompt = "1から20までの間の数で、5で終わる数は？"

prompt_formatted = f"""[INST] <<SYS>>あなたは誠実で優秀な日本人のアシスタントです。<</SYS>> {prompt} [/INST]"""


# 推論の実行
for jsn in llm(
    prompt_formatted,
    max_tokens=128,
    temperature=0.0001,
    stop=None,
    echo=True,
    stream=True,
):
    print(jsn["choices"][0]["text"], sep="", end="")
print("")

  富士山の標高は3,776メートルです。



llama_print_timings:        load time =     863.28 ms
llama_print_timings:      sample time =       9.91 ms /    20 runs   (    0.50 ms per token,  2017.35 tokens per second)
llama_print_timings: prompt eval time =     863.22 ms /    45 tokens (   19.18 ms per token,    52.13 tokens per second)
llama_print_timings:        eval time =    2858.39 ms /    19 runs   (  150.44 ms per token,     6.65 tokens per second)
llama_print_timings:       total time =    3784.78 ms


In [5]:
from app.llama2cpp.component.llama2cpp import LlamaCppCustom


n_gqa = 8 if "70b" in model_file else 1
llm = LlamaCppCustom(
    model_path=model_file,
    n_ctx=512,
    temperature=0.0001,
    max_tokens=1024,
    n_gqa=n_gqa,
    n_gpu_layers=-1,
    verbose=True,
    streaming=True,
)

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from ../data/ELYZA-japanese-Llama-2-7b-fast-instruct-q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = ELYZA-japanese-Llama-2-7b-fast-instruct
llama_model_loader: - kv   2:      general.source.huggingface.repository str              = elyza/ELYZA-japanese-Llama-2-7b-fast-...
llama_model_loader: - kv   3:                   llama.tensor_data_layout str              = Meta AI original pth
llama_model_loader: - kv   4:                       llama.context_length u32              = 4096
llama_model_loader: - kv   5:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   6:                          llama.bloc

In [8]:
prompt = "富士山の高さは？正確に"
# prompt = "1から20までの間の数で、5で終わる数は？"

prompt_formatted = f"""[INST] <<SYS>>あなたは誠実で優秀な日本人のアシスタントです。<</SYS>> {prompt} [/INST]"""

text = ""
# async for tkn in llm.astream(input=prompt_formatted, stop=None):
for tkn in llm.stream(input=prompt_formatted, stop=None):
    print(tkn, sep="", end="")
    text += tkn
print("")

Llama.generate: prefix-match hit


  富士山の標高は3,776メートルです。



llama_print_timings:        load time =     101.97 ms
llama_print_timings:      sample time =       9.94 ms /    20 runs   (    0.50 ms per token,  2012.07 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =     757.24 ms /    20 runs   (   37.86 ms per token,    26.41 tokens per second)
llama_print_timings:       total time =     820.33 ms
